In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

Hello world


In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vaini,-21.2000,-175.2000,77.00,83,20,9.22,TO,2021-04-04 21:08:26
1,1,Falesti,47.5736,27.7092,37.99,75,4,4.00,MD,2021-04-04 21:08:26
2,2,Saint-Pierre,-21.3393,55.4781,77.00,78,0,5.75,RE,2021-04-04 21:08:26
3,3,Jalu,29.0331,21.5482,72.59,10,0,11.45,LY,2021-04-04 21:08:26
4,4,Khasan,42.4308,130.6434,37.67,54,0,4.38,RU,2021-04-04 21:08:27


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vaini,-21.2000,-175.2000,77.00,83,20,9.22,TO,2021-04-04 21:08:26
2,2,Saint-Pierre,-21.3393,55.4781,77.00,78,0,5.75,RE,2021-04-04 21:08:26
6,6,Kieta,-6.2167,155.6333,80.51,80,99,3.24,PG,2021-04-04 21:08:27
13,13,Mana,5.6592,-53.7767,79.21,84,77,9.86,GF,2021-04-04 21:08:28
16,16,Itarema,-2.9248,-39.9167,79.81,81,99,13.15,BR,2021-04-04 21:08:29
18,18,Georgetown,5.4112,100.3354,84.00,94,20,3.47,MY,2021-04-04 21:08:29
22,22,Butaritari,3.0707,172.7902,82.06,79,94,16.17,KI,2021-04-04 21:08:30
24,24,San Patricio,28.0170,-97.5169,82.40,42,75,17.27,US,2021-04-04 21:08:30
25,25,Rio Grande,-32.0350,-52.0986,78.80,69,40,20.71,BR,2021-04-04 21:08:30
32,32,Arraial Do Cabo,-22.9661,-42.0278,77.00,78,40,4.61,BR,2021-04-04 21:08:32


In [10]:

preferred_cities_df.count()


City_ID       176
City          176
Lat           176
Lng           176
Max Temp      176
Humidity      176
Cloudiness    176
Wind Speed    176
Country       175
Date          176
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vaini,TO,77.00,-21.2000,-175.2000,
2,Saint-Pierre,RE,77.00,-21.3393,55.4781,
6,Kieta,PG,80.51,-6.2167,155.6333,
13,Mana,GF,79.21,5.6592,-53.7767,
16,Itarema,BR,79.81,-2.9248,-39.9167,
18,Georgetown,MY,84.00,5.4112,100.3354,
22,Butaritari,KI,82.06,3.0707,172.7902,
24,San Patricio,US,82.40,28.0170,-97.5169,
25,Rio Grande,BR,78.80,-32.0350,-52.0986,
32,Arraial Do Cabo,BR,77.00,-22.9661,-42.0278,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [14]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'ATtYBwLn79WBKkWXpFqrzsFCkHASh8XIoOB2kvmwTxHK_hUR0S-Yo0eErtUTva7CfQmkLLdlTERvh37Ck3sZRahR3RvOLpGtgbCMD3FcRDbR3ury7dpKtJPpaHR9PQhhZPEqLX2349__JuWU7zjSXcK0oWLODWr1uYTmEUcqa_UPRk9hh_eI_UWlBvOqLpWNDce0KfZIUMI4sDA5PilYc0O8VyG88O9e8IWNXp0dyEQmZEAjZHxMb-CBne85Gn-4OG85Qj8e72XeNUfHXH-iQKBZzNcO_8-ZR_SXW3Edfh09wcsojQdLTdvefY6bU17sauG43r7AQYJsvmPtQlIuNQcH9oyQesC5HYJjxaM7o7sb__yEzOR4F9P9A9gjNJM_rkesBVUiJ36EnaTD5xLdFAPCQ5LoTBm0ZA6II67dkQgpaJMa2MaKMENorh_F',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594393802915,
      'lng': 2.354352980291503},
     'southwest': {'lat': 48.8567414197085, 'lng': 2.351655019708499}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Hôtel Duo',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3840,
     'html_attributions': ['<a href="https://

In [15]:
hotels["results"][0]["name"]

'Hôtel Duo'

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
try: 
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [17]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vaini,TO,77.00,-21.2000,-175.2000,
2,Saint-Pierre,RE,77.00,-21.3393,55.4781,
6,Kieta,PG,80.51,-6.2167,155.6333,
13,Mana,GF,79.21,5.6592,-53.7767,
16,Itarema,BR,79.81,-2.9248,-39.9167,
...,...,...,...,...,...,...
530,Pemba,MZ,82.40,-12.9740,40.5178,
535,North Branch,US,77.00,45.5113,-92.9802,
539,Voh,NC,75.78,-20.9667,164.7000,
543,Gambela,ET,83.48,8.2500,34.5833,


In [18]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [23]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))